In [1]:
import sys
import numpy as np
sys.path.append('../')
from deepod.models.time_series import COUTA
from load_data import load_data
from deepod.metrics import ts_metrics
from deepod.metrics import point_adjustment 

In [2]:
train_imu,train_audio,train_spec,val_imu,val_audio,val_spec,val_imu_normal,val_audio_normal,val_spec_normal = load_data()

train_combine = np.concatenate([np.mean(train_audio,axis=-1),train_imu],axis=1)

test_audio_all = np.concatenate([val_audio_normal,val_audio],axis=0)
test_imu_all   = np.concatenate([val_imu_normal,val_imu],axis=0)
test_combine = np.concatenate([np.mean(test_audio_all,axis=-1),test_imu_all],axis=1)

labels_all = np.concatenate([np.zeros(val_audio_normal.shape[0]),np.ones(val_audio.shape[0])])

In [6]:
clf_imu = COUTA(epochs=50,seq_len=20,stride=20)
clf_imu.fit(train_imu)
clf_audio = COUTA(epochs=50,seq_len=20,stride=20)
clf_audio.fit(np.mean(train_audio,axis=-1))
clf_all = COUTA(epochs=50,seq_len=20,stride=20)
clf_all.fit(train_combine)

/home/iot/anaconda3/envs/ai/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


|>>> epoch: 01  |   loss: 0.130608, loss_oc: 0.057185, val_loss: 0.025547
|>>> epoch: 02  |   loss: 0.088472, loss_oc: 0.017729, val_loss: 0.012168
|>>> epoch: 03  |   loss: 0.075245, loss_oc: 0.009150, val_loss: 0.006699
|>>> epoch: 04  |   loss: 0.066701, loss_oc: 0.005166, val_loss: 0.003878
|>>> epoch: 05  |   loss: 0.060438, loss_oc: 0.003051, val_loss: 0.002357
|>>> epoch: 06  |   loss: 0.055962, loss_oc: 0.001886, val_loss: 0.001489
|>>> epoch: 07  |   loss: 0.052987, loss_oc: 0.001195, val_loss: 0.000929
|>>> epoch: 08  |   loss: 0.051123, loss_oc: 0.000742, val_loss: 0.000577
|>>> epoch: 09  |   loss: 0.050186, loss_oc: 0.000465, val_loss: 0.000341
|>>> epoch: 10  |   loss: 0.050818, loss_oc: 0.000296, val_loss: 0.000252
|>>> epoch: 11  |   loss: 0.049621, loss_oc: 0.000231, val_loss: 0.000220
|>>> epoch: 12  |   loss: 0.049416, loss_oc: 0.000212, val_loss: 0.000196
|>>> epoch: 13  |   loss: 0.049368, loss_oc: 0.000197, val_loss: 0.000198
|>>> epoch: 14  |   loss: 0.047994, lo

In [9]:
scores_audio      = clf_audio.decision_function(np.mean(test_audio_all,axis=-1))
scores_imu        = clf_imu.decision_function(test_imu_all)
scores_all        = clf_all.decision_function(test_combine)

In [10]:
#auc, mean precision, F1, percision, recall
eval_metrics_audio = ts_metrics(labels_all, scores_audio)
eval_metrics_imu = ts_metrics(labels_all, scores_imu)
result_audio = eval_metrics_audio
result_imu = eval_metrics_imu
eval_metrics_all = ts_metrics(labels_all, scores_all)
result_all = eval_metrics_all

print(result_audio)
print(result_imu)
print(result_all)

(0.8593749999999999, 0.8000551091340012, 0.7495488536835283, 0.7086614173228346, 0.7954545454545454)
(0.8116963217703349, 0.8151401776186022, 0.7240599953819229, 0.8208, 0.6477272727272727)
(0.8471012551501861, 0.8281063228920604, 0.7536962330846577, 0.8069164265129684, 0.7070707070707071)
